In [1]:
#Setup
import os

# from arcgis.gis import GIS
# from arcgis.features import FeatureLayer
# from arcgis.features import FeatureLayerCollection
# from arcgis.mapping import WebMap
# from arcgis.geometry import Geometry
#gis = GIS("home")

import arcpy


In [2]:
#environment variables
arcpy.env.overwriteOutput = True

In [3]:
#get path to LRM report
lrmPath = 'D:/jplatt/projects/TNC_Lands/data/Fee_CE_Assessments/NAR Fee & Eas report 12-21-21.xlsx'

In [4]:
lrmSheetName = 'Sheet1'

In [5]:
gdbPath = 'D:/jplatt/projects/TNC_Lands/data/TNC_Lands_working.gdb'

# gdbPath = 'https://services.arcgis.com/F7DSX1DSNSiWmOqh/arcgis/rest/services/TNC_Lands_NA/FeatureServer'
# feature_service_id = '88c9d0598cd345b6a098c60866636bd7'
# source_fl_item = gis.content.get(feature_service_id)
# source_flc = FeatureLayerCollection.fromitem(source_fl_item)


In [6]:
#load domains - used to check that all fields that have a domain have valid entries

#interest codes
tncInt = ['Fee Ownership', 'Conservation Easement', 'Deed Restrictions', 'Deed Restrictions - MonReq', 'Deed Restrictions - NoMon', 
'Management Lease or greement', 'Timber Lease or Agreement', 'Grazing Lease', 'Grazing Permit', 'Life Estate', 
'Right of Way Tract', 'Access Right of Way', 'Assist', 'Assist - Fee Ownership', 'Assist - Conservation Easement', 
'Assist - Deed Restriction', 'Transfer', 'Transfer - Fee Ownership', 'Transfer - Conservation Easement', 
'Transfer - Deed Restriction', 'Transfer - Life Estate', 'Transfer - Management Lease or Agreement', 'Transfer - Agreement']

#GAP codes
tnc_GAPCAT = {'1': 'managed for biodiversity – disturbance events proceed or are mimicked',
              '2': 'managed for biodiversity – disturbance events suppressed',
              '3': 'managed for multiple uses – subject to extractive (eg. mining or logging) or OHV use',
              '4': 'no known mandate for biodiversity protection'}

#and others as required

In [7]:
#create table view on LRM report
if arcpy.Exists(os.path.join(gdbPath, 'lrmReport')):
    try:
        arcpy.Delete_management(os.path.join(gdbPath, 'lrmReport'))
    except arcpy.ExecuteError:
        print(arcpy.GetMessages(2))
    except Exception as e:
        print(e.args[0])

arcpy.ExcelToTable_conversion(lrmPath, os.path.join(gdbPath, 'lrmReport'), lrmSheetName)

<Result 'D:\\jplatt\\projects\\TNC_Lands\\data\\TNC_Lands_working.gdb\\lrmReport'>

In [8]:
indexes = arcpy.ListIndexes(os.path.join(gdbPath, 'lrmReport'))
try:
    if 'indx_trID' not in indexes: 
        arcpy.AddIndex_management(os.path.join(gdbPath, 'lrmReport'), ['LRM_Tract_ID'], 'indx_trID')
    else:
        arcpy.RemoveIndex_management(os.path.join(gdbPath, 'lrmReport'), ['LRM_Tract_ID'])
        arcpy.AddIndex_management(os.path.join(gdbPath, 'lrmReport'), ['LRM_Tract_ID'], 'indx_trID')
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(e.args[0])


In [9]:
#set path to TNC Lands data
tncLandsPath = os.path.join(gdbPath, 'TNC_Lands_Base')

In [ ]:
#create search cursor on TNC Lands table
#tractCursor = arcpy.UpdateCursor(os.path.join(gdbPath, tncLandsPath))
fieldlist = arcpy.ListFields(os.path.join(gdbPath, tncLandsPath))
for item in fieldlist:
    print(item.name)


In [ ]:
#iterate over the LRM report
#create search cursor on LRM report
state = 'SD'
tractIDs = [1000000,1000001,1000016,1000025,1000128,1000193]
for tractid in tractIDs:
    lrmCursor = arcpy.SearchCursor(os.path.join(gdbPath, 'lrmReport'), f"Primary_Geocode = '{state}' AND LRM_Tract_ID = {tractid}")
    for lrmRow in lrmCursor:
        print(f"{lrmRow.getValue('LRM_Tract_ID')}\t{lrmRow.getValue('Tract_Name')}")
        #select matching TNC Lands records
        try:
            tractCursor = arcpy.UpdateCursor(os.path.join(gdbPath, tncLandsPath),  f"LRM_TR_ID = {tractid}")
        except arcpy.ExecuteError:
            print(arcpy.GetMessages(2))
        except Exception as e:
            print(e.args[0])
        for trRow in tractCursor:
            print(f"{trRow.getValue('LRM_TR_ID')}\t{trRow.getValue('LRM_TR_NA')}")


In [12]:
#start checking LRM fields
lrmCursor = arcpy.SearchCursor(os.path.join(gdbPath, 'lrmReport'))
for lrmRow in lrmCursor:
    print(f"{lrmRow.getValue('LRM_Tract_ID')}\t{lrmRow.getValue('Tract_Name')}")
    #select matching TNC Lands records
    try:
        tractCursor = arcpy.UpdateCursor(os.path.join(gdbPath, tncLandsPath),  f"LRM_TR_ID = {tractid}")
    except arcpy.ExecuteError:
        print(arcpy.GetMessages(2))
    except Exception as e:
        print(e.args[0])
    for trRow in tractCursor:
        print(f"{trRow.getValue('LRM_TR_ID')}\t{trRow.getValue('LRM_TR_NA')}")

1000000	Institute of Range and the American Mustang/IRAM 2 (TNC EAS)
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000001	Wesleyan School (TNC)
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000004	Day Companies 1
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000005	Day Companies 6
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000006	Day Companies 3
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000007	Creative Investments
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000012	Neuman (TNC)
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000013	Kindschi (Ivan) Estate
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000015	Barrows (TNC 45% UDV/Barrows 55% UDV)
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000016	Hutton (TNC)
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000017	Butler 1
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000018	Johnston 4
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000025	South Dakota Office of School & Public Lands (fka South Dakota Parks & Wildlife Foundation)
1000193	Teuber (Fka Bhl Capital Corp. 1)
1000026	Howa

KeyboardInterrupt: 